# Job Loader

This notebook is used to load jobs in this repo to the `wandb/jobs` public project.
- You will need to be logged into wandb and have access to the `wandb` entity.

## Setup

Kill ports ahead of spinning up containers (you may need to restart docker)

In [3]:
!lsof -i TCP:3307 | grep LISTEN | awk '{print $2}' | xargs kill -9
!lsof -i TCP:8000 | grep LISTEN | awk '{print $2}' | xargs kill -9
!open /Applications/Docker.app

In [ ]:
import wandb
from pathlib import Path

In [ ]:
%%capture
%env WANDB_API_KEY {wandb.api.api_key}
%env WANDB_ENTITY wandb
%env WANDB_PROJECT jobs

## Python jobs

In [4]:
%env WANDB_RUN_GROUP python

env: WANDB_RUN_GROUP=python


Note: The SQL Query job depends on access to a database.  You can load this dummy database with the snippet below:

In [5]:
!docker run -p 3307:3306 -d sakiladb/mysql:latest
%env MYSQL_USER sakila
%env MYSQL_PASSWORD p_ssW0rd

fc792be730733e8b72a7e4aba7e84a6c85148460ae8c6088bd6a23514b32029c
env: MYSQL_USER=sakila
env: MYSQL_PASSWORD=p_ssW0rd


Run python jobs as usual:

In [6]:
python_jobs = list(Path('jobs').glob('**/*job.py'))
python_jobs

[PosixPath('jobs/hello_world/job.py'),
 PosixPath('jobs/sql_query_artifact/job.py'),
 PosixPath('jobs/sql_query_table/job.py'),
 PosixPath('jobs/msft_teams_webhook/job.py'),
 PosixPath('jobs/http_webhook/job.py'),
 PosixPath('jobs/github_actions_workflow_dispatch/job.py')]

In [ ]:
for job in python_jobs:
    %env WANDB_NAME {job.parent.name}
    %env WANDB_JOBS_REPO_CONFIG {job.parent/'config.yml'}
    !python {job}

## Docker jobs
- These jobs touch AWS, so they mount the `.aws` directory.
- If you need to see the literal command, prepend `set -x &&` to the shell command

In [ ]:
%env WANDB_RUN_GROUP docker

## Sagemaker Endpoints job

In [ ]:
%env WANDB_NAME deploy_to_sagemaker_endpoints
%env WANDB_JOBS_REPO_CONFIG config_tensorflow.yml

!docker build -t $WANDB_NAME jobs/deploy_to_sagemaker_endpoints && \
docker run \
   -v $HOME/.aws:/root/.aws:ro \
   -e WANDB_API_KEY=$WANDB_API_KEY \
   -e WANDB_ENTITY=$WANDB_ENTITY \
   -e WANDB_PROJECT=$WANDB_PROJECT \
   -e WANDB_NAME=$WANDB_NAME \
   -e WANDB_RUN_GROUP=$WANDB_RUN_GROUP \
   -e WANDB_JOBS_REPO_CONFIG=$WANDB_JOBS_REPO_CONFIG \
   $WANDB_NAME

env: WANDB_NAME=deploy_to_sagemaker_endpoints
env: WANDB_JOBS_REPO_CONFIG=config_tensorflow.yml
[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 343B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.9-buster       0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 343B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring c

## Nvidia Triton Job

This job requires a running Triton Server.  You can start one with this snippet

In [ ]:
# you may need this export on M1
# related: https://github.com/keras-team/keras-tuner/issues/317#issuecomment-640181692
%env LD_PRELOAD="/usr/lib/aarch64-linux-gnu/libgomp.so.1"

!docker build -t tritonserver-wandb jobs/deploy_to_nvidia_triton/server && \
docker run \
  -v $HOME/.aws:/root/.aws:ro \
  -p 8000:8000 \
  --rm --net=host -d \
  tritonserver-wandb

env: LD_PRELOAD="/usr/lib/aarch64-linux-gnu/libgomp.so.1"
[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 208B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for nvcr.io/nvidia/tritonserver:22.11-py3     0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 208B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                            

Then launch the job

In [ ]:
%env WANDB_NAME deploy_to_nvidia_triton
%env WANDB_JOBS_REPO_CONFIG config_tensorflow.yml

!docker build -t $WANDB_NAME jobs/deploy_to_nvidia_triton/deployer && \
docker run \
   -v $HOME/.aws:/root/.aws:ro \
   -e WANDB_API_KEY=$WANDB_API_KEY \
   -e WANDB_ENTITY=$WANDB_ENTITY \
   -e WANDB_PROJECT=$WANDB_PROJECT \
   -e WANDB_NAME=$WANDB_NAME \
   -e WANDB_RUN_GROUP=$WANDB_RUN_GROUP \
   -e WANDB_JOBS_REPO_CONFIG=$WANDB_JOBS_REPO_CONFIG \
   --rm --net=host \
   $WANDB_NAME

env: WANDB_NAME=deploy_to_nvidia_triton
env: WANDB_JOBS_REPO_CONFIG=config_tensorflow.yml
[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 74B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.9-buster       0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 74B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context

## Nvidia Tensor RT Conversion Job
This job requires a GPU.

In [ ]:
%env WANDB_NAME optimize_with_nvidia_tensorrt

!docker build -t $WANDB_NAME jobs/optimize-with-tensor-rt && \
docker run \
    --gpus all \
    --runtime=nvidia \
    -e WANDB_API_KEY=$WANDB_API_KEY \
    -e WANDB_ENTITY=$WANDB_ENTITY \
    -e WANDB_PROJECT=$WANDB_PROJECT \
    -e WANDB_RUN_GROUP=$WANDB_RUN_GROUP \
    -e WANDB_NAME=$WANDB_NAME \
    $WANDB_NAME

env: WANDB_NAME=optimize_with_nvidia_tensorrt
unable to prepare context: path "jobs/optimize-with-tensor-rt" not found
